In [1]:
from models import BiSeNetv2
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the ICNet model with the same configuration as during training
net = BiSeNetv2(num_class=19, n_channel=3, act_type='relu', use_aux=True)

# Load the checkpoint
checkpoint_path = 'C:/Users/avshinde/Downloads/New folder/New folder/bisenetv2-aux.pth'  # Replace with the actual path to your checkpoint file
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Add map_location argument if needed
print(checkpoint.keys())

# Load the model state_dict
net.load_state_dict(checkpoint['state_dict'])

net=net.to(device)

# Ensure the model is in evaluation mode
net.eval()

dict_keys(['cur_epoch', 'best_score', 'state_dict', 'optimizer', 'scheduler'])


BiSeNetv2(
  (detail_branch): DetailBranch(
    (0): ConvBNAct(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Activation(
        (activation): ReLU()
      )
    )
    (1): ConvBNAct(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Activation(
        (activation): ReLU()
      )
    )
    (2): ConvBNAct(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Activation(
        (activation): ReLU()
      )
    )
    (3): ConvBNAct(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [4]:
total_params = 0
non_zero_params = 0
zero_params = 0

for name, param in net.state_dict().items():
        total_params += param.numel()
        non_zero_params += torch.count_nonzero(param).item()
        zero_params += (param.numel() - torch.count_nonzero(param)).item()
print(f"Total Parameters: {total_params}")
print(f"Non-Zero Parameters: {non_zero_params}")
print(f"Zero Parameters: {zero_params}")

Total Parameters: 2576409
Non-Zero Parameters: 2576409
Zero Parameters: 0


### Prune

In [5]:
from prune import prune_icnet
prune_icnet(net, method='std', s=0.25)

In [6]:
# If needed, remove pruning
from torch.nn.utils import prune

for name, module in net.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        prune.remove(module, 'weight')

In [7]:
total_params = 0
non_zero_params = 0
zero_params = 0

for name, param in net.state_dict().items():
        total_params += param.numel()
        non_zero_params += torch.count_nonzero(param).item()
        zero_params += (param.numel() - torch.count_nonzero(param)).item()
print(f"Total Parameters: {total_params}")
print(f"Non-Zero Parameters: {non_zero_params}")
print(f"Zero Parameters: {zero_params}")

Total Parameters: 2576409
Non-Zero Parameters: 1951209
Zero Parameters: 625200


In [7]:
# # Print the names and values of the parameters
# for name, param in net.named_parameters():
#     print(f'Parameter name: {name}')
#     print(f'Parameter shape: {param.shape}')
#     print(f'Parameter data:\n{param.data}\n')

In [8]:
# Create a new model to store the pruned weights
pruned_model = BiSeNetv2(num_class=19, n_channel=3, act_type='relu', use_aux=True)
pruned_model.load_state_dict(net.state_dict())

<All keys matched successfully>

In [9]:
# Save the updated checkpoint
updated_checkpoint_path = 'updated_checkpoint_BiSeNetv2.pth'
checkpoint['state_dict'] = pruned_model.state_dict()
checkpoint['cur_epoch'] = 0
checkpoint['best_score'] = 0
torch.save(checkpoint, updated_checkpoint_path)

### Fine-tuning pruned weights by re-training

In [1]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 10:47] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/updated_checkpoint_BiSeNetv2.pth
[2023-12-04 10:47] Resume training from C:/Users/avshinde/Downloads/New folder/New folder/updated_checkpoint_BiSeNetv2.pth
[2023-12-04 10:47] 


######################### Config Informations #########################
dataset: cityscapes
num_class: 19
model: bisenetv2
encoder: None
decoder: None
loss_type: ohem
optimizer_type: adam
lr_policy: cos_warmup
total_epoch: 10
train_bs: 8
val_bs: 8
train_num: 2968
val_num: 500
gpu_num: 1
num_workers: 8
amp_training: False
DDP: False
kd_training: False
synBN: True
use_ema: True
use_aux: True
###############################################################

tensor(0.3224, device='cuda:0')


Epoch:2/10    |Loss:5.53    |: 100%|██████████| 371/371 [06:19<00:00,  1.02s/it] 
Validating:    |: 100%|██████████| 63/63 [02:03<00:00,  1.97s/it] 
[2023-12-04 11:04]  Epoch2 mIoU: 0.5080    | best mIoU so far: 0.3224



tensor(0.5080, device='cuda:0')


Epoch:3/10    |Loss:6.679    |:  16%|█▌        | 60/371 [01:47<09:16,  1.79s/it] 


KeyboardInterrupt: 

##### Original mIoU: 73.73


##### Obtained mIoU: 66.18

### Prediction on the pruned weights

In [1]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 09:27] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/updated_checkpoint.pth
[2023-12-04 09:27] 
Start predicting...

100%|██████████| 38/38 [09:14<00:00, 14.58s/it]


### Video Output

In [2]:
import cv2
import os
from natsort import natsorted

# Directory containing the predicted segmentation images
image_folder = 'C:/Users/avshinde/Downloads/New folder/New folder/save'

# Output video file name
video_name = 'output_video_mainz.avi'

# Get the list of image files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith("blend.png")]
images = natsorted(images)

# Get image dimensions from the first image
img = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = img.shape

# Define the desired frame rate (adjust as needed)
desired_frame_rate = 1 # Frames per second

# Create a VideoWriter object with the desired frame rate
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), desired_frame_rate, (width, height))

# Loop through the images and write each frame to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the VideoWriter object
video.release()

In [3]:
import cv2
import os
from natsort import natsorted
import imageio

# Directory containing the predicted segmentation images
image_folder = 'C:/Users/avshinde/Downloads/New folder/New folder/save'

# Output GIF file name
gif_name = 'output_video_mainz.gif'

# Get the list of image files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith("blend.png")]
images = natsorted(images)

# Get image dimensions from the first image
img = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = img.shape

# Define the desired frame rate (adjust as needed)
desired_frame_rate = 0.7 # Frames per second

# Create a list to store frames
frames = []

# Loop through the images and append each frame to the list
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    # Ensure that the color channels are in the correct order (BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

# Save the frames as a GIF using imageio
imageio.mimsave(gif_name, frames, duration=1/desired_frame_rate)

# Note: The duration parameter specifies the time each frame is displayed in seconds (1/desired_frame_rate).

print(f'GIF saved as {gif_name}')


GIF saved as output_video_mainz.gif


## Validating the original model

In [1]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 10:27] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/bisenetv2-aux.pth
[2023-12-04 10:27] Resume training from C:/Users/avshinde/Downloads/New folder/New folder/bisenetv2-aux.pth
[2023-12-04 10:27] 


######################### Config Informations #########################
dataset: cityscapes
num_class: 19
model: bisenetv2
encoder: None
decoder: None
loss_type: ohem
optimizer_type: adam
lr_policy: cos_warmup
total_epoch: 10
train_bs: 8
val_bs: 8
train_num: 2968
val_num: 500
gpu_num: 1
num_workers: 8
amp_training: False
DDP: False
kd_training: False
synBN: True
use_ema: True
use_aux: True
#######################################################################


[2023-12-04 10:27] 

tensor(0.7373, device='cuda:0')


##### mIoU: 73.73

### Predicting on the original model

In [1]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 10:32] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/bisenetv2-aux.pth
[2023-12-04 10:32] 
Start predicting...

100%|██████████| 38/38 [09:09<00:00, 14.45s/it]


#### Video Output

In [1]:
import cv2
import os
from natsort import natsorted

# Directory containing the predicted segmentation images
image_folder = 'C:/Users/avshinde/Downloads/New folder/New folder/save/mainz_unpruned_bisenetv2/blend'

# Output video file name
video_name = 'output_video_mainz_unpruned.avi'

# Get the list of image files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images = natsorted(images)

# Get image dimensions from the first image
img = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = img.shape

# Define the desired frame rate (adjust as needed)
desired_frame_rate = 1 # Frames per second

# Create a VideoWriter object with the desired frame rate
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), desired_frame_rate, (width, height))

# Loop through the images and write each frame to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the VideoWriter object
video.release()